In [9]:
#@title run.sh shell script
%%file run.sh
#!/bin/bash
# compile and run
g++ -std=c++11 -fopenmp -o ./$1 $1.cpp && ./$1

# $? is the value returned by the command we just ran.
if [ $? -eq 0 ]
then
    echo "PASS"
else
    echo "FAIL"
fi

Overwriting run.sh


In [10]:
#@title This cells makes the script executable by the user
!chmod u+x ./run.sh

# Bitonic sort

Modify the code below so that the code runs in parallel using OpenMP.

In [11]:
%%file bitonic_sort.cpp

#include <omp.h>

#include <algorithm>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <utility>
#include <vector>

using namespace std;

typedef std::vector<int> vint;

#ifndef NDEBUG
#define LogInfo(M, ...)                                            \
  fprintf(stderr, "[INFO] (%s:%d:%d) " M "\n", __FILE__, __LINE__, \
          omp_get_thread_num(), ##__VA_ARGS__)
#else
#define LogInfo(...)
#endif

void BitonicSortSeq(int start, int length, vint &seq, bool up);
void BitonicSortPar(int start, int length, vint &seq, bool up, int chunk);
void PrintSequence(int n, vint &seq, int s);

int main() {
  // n must be a power of 2
#ifndef NDEBUG
  const int n = 1 << 4;
#else
  const int n = 1 << 23;
#endif
  printf("Size of array: %d\n", n);

  vint seq(n);
  std::generate(seq.begin(), seq.end(), [] { return rand() % 100; });

  vint seq_debug = seq;
  // Sort seq_debug
  sort(seq_debug.begin(), seq_debug.end());

  // print the original sequence
  PrintSequence(n, seq, n);

  omp_set_num_threads(4);

  int n_threads;
#pragma omp parallel
#pragma omp single
  n_threads = omp_get_num_threads();

  // making sure input is okay
  if (n < n_threads * 2) {
    printf(
        "The size of the sequence is less than 2 * the number of processes.\n");
    exit(0);
  }

  // the size of sub parts
  // chunk = n / n_threads;
  int chunk = 1;

  while (chunk <= (n / n_threads) >> 1) {
    chunk <<= 1;
  }

  assert(n_threads <= n / chunk);
  assert(n % chunk == 0);

  // n_threads <= n/chunk
  // chunk <= n/n_threads

  printf("Size of chunks: %d\n", chunk);
  printf("Number of chunks: %d\n", (n + chunk - 1) / chunk);
  printf("Number of threads: %d\n", n_threads);

  // start
  double begin_time, elapsed_time; /* for checking/testing timing */
  begin_time = omp_get_wtime();

  // Part 1: merge chunks with small length
  for (int i = 2; i <= chunk; i <<= 1) {
    // We have many chunks so we can parallelize the j loop
#pragma omp parallel for
    for (int j = 0; j < n; j += i) {
      bool up = ((j / i) % 2 == 0);
      // This step is sequential
      BitonicSortSeq(j, i, seq, up);
    }

    LogInfo("Sort of length %d", i);
    PrintSequence(n, seq, i);
  }

  // Part 2: merge all the larger chunks
  // We have few chunks but long ones
  for (int i = chunk << 1; i <= n; i <<= 1) {
    for (int j = 0; j < n; j += i) {
      bool up = ((j / i) % 2 == 0);
      BitonicSortPar(j, i, seq, up, chunk);
      LogInfo("Parallel sort of length %d [%d]", i, j / i);
      PrintSequence(n, seq, i);
    }

    // BitonicSortPar is calling itself recursively.
    // At each pass the chunks get twice smaller.
    // So at some point, the chunks are very small and
    // we need to run the sequential algorithm again.
    // See the line
    // if (split_length > chunk)
    // in BitonicSortPar.
    // For that reason, we have again a parallel loop over j
#pragma omp parallel for
    for (int j = 0; j < n; j += chunk) {
      bool up = ((j / i) % 2 == 0);
      BitonicSortSeq(j, chunk, seq, up);
    }

    LogInfo("Sequential sort of length %d", chunk);
    PrintSequence(n, seq, chunk);
  }

  // end
  elapsed_time = omp_get_wtime() - begin_time;

  // print the sorted sequence
  LogInfo("Calculation complete");
  PrintSequence(n, seq, n);

  assert(equal(seq.begin(), seq.end(), seq_debug.begin()));

  LogInfo("Result of bitonic sort is correct and has passed the test.");

  printf("Elapsed time = %.2f sec, p T_p = %.2f.\n", elapsed_time,
         n_threads * elapsed_time);

  return 0;
}

void BitonicSortPar(int start, int length, vint &seq, bool up, int chunk) {
  if (length == 1) {
    return;
  }

  if (length % 2 != 0) {
    printf("The length of a (sub)sequence is not divisible by 2.\n");
    exit(0);
  }

  const int split_length = length / 2;

// bitonic split
// split_length is large
#pragma omp parallel for
  for (int k = start; k < start + split_length; k++) {
    if (up) {
      if (seq[k] > seq[k + split_length]) {
        swap(seq[k], seq[k + split_length]);
      }
    } else if (seq[k] < seq[k + split_length]) {
      swap(seq[k], seq[k + split_length]);
    }
  }

  if (split_length > chunk) {
    // chunk is the size of the chunks
    // If the chunks are small, we revert to the serial algorithm
    // BitonicSortSeq
    BitonicSortPar(start, split_length, seq, up, chunk);
    BitonicSortPar(start + split_length, split_length, seq, up, chunk);
  }
}

void BitonicSortSeq(int start, int length, vint &seq, bool up) {
  if (length == 1) {
    return;
  }

  if (length % 2 != 0) {
    printf("The length of a (sub)sequence is not divisible by 2.\n");
    exit(0);
  }

  const int split_length = length / 2;

  // bitonic split
  for (int k = start; k < start + split_length; k++) {
    if (up) {
      if (seq[k] > seq[k + split_length]) {
        swap(seq[k], seq[k + split_length]);
      }
    } else if (seq[k] < seq[k + split_length]) {
      swap(seq[k], seq[k + split_length]);
    }
  }

  // recursive sort
  BitonicSortSeq(start, split_length, seq, up);
  BitonicSortSeq(start + split_length, split_length, seq, up);
}

void PrintSequence(int n, vint &seq, int s) {
#ifndef NDEBUG
  int color = 0;
  printf("[");

  for (int i = 0; i < n; i++) {
    if (i % s == 0) {
      color = 1 - color;
    }

    if (color == 1) {
      printf("\x1B[0m%3d", seq[i]);
    } else {
      printf("\x1B[31m%3d", seq[i]);
    }
  }

  printf("\x1B[0m]\n");
#endif
}


Overwriting bitonic_sort.cpp


In [12]:
!./run.sh bitonic_sort

Size of array: 16
[ 83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26]
Size of chunks: 4
Number of chunks: 4
Number of threads: 4
[INFO] (bitonic_sort.cpp:92:0) Sort of length 2
[ 83 86 77 15 35 93 92 86 21 49 62 27 59 90 63 26]
[INFO] (bitonic_sort.cpp:92:0) Sort of length 4
[ 15 77 83 86 93 92 86 35 21 27 49 62 90 63 59 26]
[INFO] (bitonic_sort.cpp:102:0) Parallel sort of length 8 [0]
[ 15 77 83 35 93 92 86 86 21 27 49 62 90 63 59 26]
[INFO] (bitonic_sort.cpp:102:0) Parallel sort of length 8 [1]
[ 15 77 83 35 93 92 86 86 90 63 59 62 21 27 49 26]
[INFO] (bitonic_sort.cpp:120:0) Sequential sort of length 4
[ 15 35 77 83 86 86 92 93 90 63 62 59 49 27 26 21]
[INFO] (bitonic_sort.cpp:102:0) Parallel sort of length 16 [0]
[ 15 27 26 21 49 35 62 59 86 63 77 83 90 86 92 93]
[INFO] (bitonic_sort.cpp:120:0) Sequential sort of length 4
[ 15 21 26 27 35 49 59 62 63 77 83 86 86 90 92 93]
[INFO] (bitonic_sort.cpp:128:0) Calculation complete
[ 15 21 26 27 35 49 59 62 63 77 83 86 86 90 92 93]
[INFO] (